In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("/home/hana/sonnh/kaggle-cassava/dataset/train_mix/mix.csv")

In [18]:
df.groupby('label').count()

,index,image_id,file
label,,,
0,1491,1491,1491
1,3476,3476,3476
2,3017,3017,3017
3,15462,15462,15462
4,2889,2889,2889


In [26]:
[1491/1491, round(1491/3476, 2), round(1491/3017, 2), round(1491/15462, 2), round(1491/2889, 2)]

[1.0, 0.43, 0.49, 0.1, 0.52]

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
root_path = "/content/gdrive/My Drive/NFL/classification/"

In [ ]:
!pip install catalyst
!pip install efficientnet_pytorch

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
import albumentations.pytorch as AT
from catalyst.data.sampler import BalanceClassSampler
from data import HelmetDataset
from efficientnet_pytorch import EfficientNet
from losses import FocalBCEWithLogitsLoss
from engines import trainer_augment

In [ ]:
df = pd.read_csv(root_path + "clf.csv")
folds = 5
print("Folds: {}\n".format(folds))

In [ ]:
model_name = "efficientnet-b4"
batch_size, num_workers = 128, 4
start_epoch, total_epochs = 1, 30
device, device_ids = torch.device("cuda:0"), [0]

In [ ]:
for fold in range(1, folds + 1):
    head = "fold {:2}/{:2}".format(fold, folds)
    print(head + "\n" + "-"*(len(head)))

    train_df = df[df["fold"] != fold].reset_index(drop=True)
    train_transform = A.Compose([
        A.Resize(height=128, width=128),
        A.IAAAffine(translate_percent=(-0.0625, 0.0625), rotate=(-180, 180), shear=(-10, 10), order=[0, 1]),
        A.HorizontalFlip(),
        A.RandomBrightnessContrast(),
        A.HueSaturationValue(),
        A.Normalize(),
        AT.ToTensor()
    ])
    train_loader = torch.utils.data.DataLoader(
        HelmetDataset(
            df=train_df,
            image_folder=root_path + "images_100/",
            image_transform=train_transform
        ),
        batch_size=batch_size, num_workers=num_workers, drop_last=True,
        sampler=BalanceClassSampler(list(train_df["binary_target"].values), mode="downsampling")
    )

    eval_df = df[df["fold"] == fold].reset_index(drop=True)
    eval_transform = A.Compose([
        A.Resize(height=128, width=128),
        A.Normalize(),
        AT.ToTensor()
    ])
    eval_loader = torch.utils.data.DataLoader(
        HelmetDataset(
            df=eval_df,
            image_folder=root_path + "images_100/",
            image_transform=eval_transform
        ),
        batch_size=batch_size, num_workers=num_workers, drop_last=True
    )

    loaders = {
        "train": train_loader,
        "eval": eval_loader
    }

    augment_prob = 0.5

    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(model._fc.in_features, 1)
    criterion = FocalBCEWithLogitsLoss(alpha=0.75, gamma=2)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)

    trainer_augment(loaders, augment_prob, model, criterion, optimizer, lr_scheduler, 
                    start_epoch, total_epochs, device, device_ids, save_path=root_path + "checkpoints/{}_fold-{}.pt".format(model_name, fold))